<img src="../assets/a_type_readme.gif" style="float:right ; margin: 10px ; width:300px;"> 
<h1><left>NLP Project</left></h1>
<h4><left>Using Natural Language Processing to better understand Depression & Anxiety</left></h4>
___

## 3. Analysis

In [33]:
import numpy as np
from numpy import core, array
assert np.__version__ == "1.19.5"

import pandas as pd

import seaborn as sns
sns.set_style("darkgrid")

import sentencepiece as spm

import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models import Word2Vec, KeyedVectors
# from gensim.models import Word2Vec
assert gensim.__version__ == "4.0.1"

from sklearn.model_selection import KFold, train_test_split
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from pickle import dump, load

from nltk import word_tokenize

import matplotlib.pyplot as plt
%matplotlib inline

from time import time 

from random import randint

import logging 

import multiprocessing
 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Embedding

In [3]:
logging.basicConfig(filename="../language_model.log",
                    format='%(asctime)s %(message)s',
                    filemode='w')
logger = logging.getLogger()

def print_time(intput_str, start_time=0):
    print("{}: {} min".format(input_str, round((time() - start_time) / 60, 2)))
    
# #Setting the threshold of logger to DEBUG
# logger.setLevel(logging.DEBUG)
  
# #Test messages
# logger.debug("Harmless debug Message")
# logger.info("Just an information")
# logger.warning("Its a Warning")
# logger.error("Did you try to divide by zero")
# logger.critical("Internet is down")

In [4]:
model_data = pd.read_csv('../data/data_for_model.csv', keep_default_na=False)
print(model_data.info())
model_data.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1930 entries, 0 to 1929
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   title                      1930 non-null   object
 1   selftext                   1930 non-null   object
 2   author                     1930 non-null   object
 3   score                      1930 non-null   int64 
 4   num_comments               1930 non-null   int64 
 5   is_anxiety                 1930 non-null   int64 
 6   url                        1930 non-null   object
 7   selftext_clean             1930 non-null   object
 8   selftext_broken_sentences  1930 non-null   object
 9   selftext_broken_words      1930 non-null   object
 10  title_clean                1930 non-null   object
 11  author_clean               1930 non-null   object
 12  megatext_clean             1930 non-null   object
dtypes: int64(3), object(10)
memory usage: 196.1+ KB
None


,title,selftext,author,score,num_comments,is_anxiety,url,selftext_clean,selftext_broken_sentences,selftext_broken_words,title_clean,author_clean,megatext_clean
0,Our most-broken and least-understood rules is ...,We understand that most people who reply immed...,SQLwitch,2319,175,0,https://www.reddit.com/r/depression/comments/d...,understand people reply immediately op invitat...,['we understand that most people who reply imm...,"['understand', 'people', 'reply', 'immediately...",broken least understood rule helper may invite...,sql witch,sql witch understand people reply immediately ...
1,"Regular Check-In Post, with important reminder...",Welcome to /r/depression's check-in post - a p...,SQLwitch,312,1136,0,https://www.reddit.com/r/depression/comments/m...,welcome r depression check post place take mom...,"[""welcome to /r/depression's check-in post - a...","['welcome', 'r', 'depression', 'check', 'post'...",regular check post important reminder private ...,sql witch,sql witch welcome r depression check post plac...
2,Low,I'm so low rn I can't even type anything coher...,RagingFlock89,263,43,0,https://www.reddit.com/r/depression/comments/n...,low rn even type anything coherent want expres...,"[""i'm so low rn i can't even type anything coh...","['low', 'rn', 'even', 'type', 'anything', 'coh...",low,raging flock 89,raging flock 89 low rn even type anything cohe...


In [5]:
data_column = "selftext_clean"
# model_data[data_column][0]
# model_data["megatext_clean"]
# sents = [eval(sent) for sent in model_data["selftext_broken_words"]]
# dp = model_data["selftext_broken_words"].tolist()[1]
# literal_eval(dp)
# model_data["megatext_clean"].to_csv(data_path, header=None, index=None, sep='\t', mode='a')

## Language Model

### Data

### Model

In [35]:
model_data

,title,selftext,author,score,num_comments,is_anxiety,url,selftext_clean,selftext_broken_sentences,selftext_broken_words,title_clean,author_clean,megatext_clean
0,Our most-broken and least-understood rules is ...,We understand that most people who reply immed...,SQLwitch,2319,175,0,https://www.reddit.com/r/depression/comments/d...,understand people reply immediately op invitat...,['we understand that most people who reply imm...,"['understand', 'people', 'reply', 'immediately...",broken least understood rule helper may invite...,sql witch,sql witch understand people reply immediately ...
1,"Regular Check-In Post, with important reminder...",Welcome to /r/depression's check-in post - a p...,SQLwitch,312,1136,0,https://www.reddit.com/r/depression/comments/m...,welcome r depression check post place take mom...,"[""welcome to /r/depression's check-in post - a...","['welcome', 'r', 'depression', 'check', 'post'...",regular check post important reminder private ...,sql witch,sql witch welcome r depression check post plac...
2,Low,I'm so low rn I can't even type anything coher...,RagingFlock89,263,43,0,https://www.reddit.com/r/depression/comments/n...,low rn even type anything coherent want expres...,"[""i'm so low rn i can't even type anything coh...","['low', 'rn', 'even', 'type', 'anything', 'coh...",low,raging flock 89,raging flock 89 low rn even type anything cohe...
3,I’m always amazed at how much energy healthy p...,When I wake up after 8 hours of decent sleep I...,cezzzie,1281,120,0,https://www.reddit.com/r/depression/comments/n...,wake 8 hour decent sleep still exhausted day e...,['when i wake up after 8 hours of decent sleep...,"['wake', '8', 'hour', 'decent', 'sleep', 'stil...",always amazed much energy healthy people,ce zzz ie,ce zzz ie wake 8 hour decent sleep still exhau...
4,30 and never lived a day in my life,I guess i have always been depressed but never...,ApprehensiveYou2385,36,5,0,https://www.reddit.com/r/depression/comments/n...,guess always depressed never really thought mu...,['i guess i have always been depressed but nev...,"['guess', 'always', 'depressed', 'never', 'rea...",30 never lived day life,apprehensive 2385,apprehensive 2385 guess always depressed never...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1925,Any tips on how not to panic during a midterm?,I have an Applied Statics midterm tomorrow. I ...,Anomalistic_Username,2,0,1,https://www.reddit.com/r/Anxiety/comments/n3p5...,applied static midterm tomorrow already failed...,['i have an applied statics midterm tomorrow.'...,"['applied', 'static', 'midterm', 'tomorrow', '...",tip panic midterm,mali stic username,mali stic username applied static midterm tomo...
1926,"I find myself apologizing really often, checki...",I've recently decided to stop smoking weed (so...,zedhenson,3,1,1,https://www.reddit.com/r/Anxiety/comments/n3n1...,recently decided stop smoking weed socially th...,"[""i've recently decided to stop smoking weed (...","['recently', 'decided', 'stop', 'smoking', 'we...",find apologizing really often checking behavior,zed henson,zed henson recently decided stop smoking weed ...
1927,I typed out my anxiety attack and thought I sh...,I recently got into a little habit where when ...,Tree-Nui-Tee,2,4,1,https://www.reddit.com/r/Anxiety/comments/n3oz...,recently got little habit thing get overwhelmi...,['i recently got into a little habit where whe...,"['recently', 'got', 'little', 'habit', 'thing'...",typed anxiety attack thought share,tree nui tee,tree nui tee recently got little habit thing g...
1928,something happened that just triggered my anxi...,i need someone to vent to please,Capzfan5,8,5,1,https://www.reddit.com/r/Anxiety/comments/n3gc...,need someone vent please,['i need someone to vent to please'],"['need', 'someone', 'vent', 'please']",something happened triggered anxiety really bad,cap z fan 5,cap z fan 5 need someone vent please something...


In [15]:
def organize_token_seqs(data, length=20+1):
    sequences = list()
    
    for record in data:
        tokens = word_tokenize(record)
        
        for i in range(length, len(tokens)):
            seq = tokens[i-length: i]
            assert len(seq) == length, (length, len(seq), seq)
            line = ' '.join(seq)
            sequences.append(line)
        
#     if len(sequences)== 0:
#         print(tokens)
#     print('Total Sequences: %d' % len(sequences))
    
    return sequences

In [16]:
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()
    
    
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

In [17]:
def build_LM_A(vocab_size, seq_length, summary=False):
    model = Sequential()
    model.add(Embedding(vocab_size, 50, input_length=seq_length))
    model.add(LSTM(100, return_sequences=True))
    model.add(LSTM(100))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(vocab_size, activation='softmax'))
    if summary:
        print(model.summary())
    
    return model

In [20]:
def language_model(label, data, n_seq_words, EPOCHS=100, BATCH=128):
    model_path = '../models/{}.language_model.h5'.format(label)
    tokenizer_path = '../models/{}.language_model.tokenizer.pkl'.format(label)
    processed_data_path = '../data/{}.language_model.processed_data.txt'.format(label)
    
    # ------------------------------ PREPARE DATA -----------------------------------
    processed_data = organize_token_seqs(data, n_seq_words)
    save_doc(processed_data, processed_data_path)
    
#     doc = load_doc(processed_data_path)
#     processed_data = doc.split('\n')
#     print("\nword sequences\n", processed_data[:5])
    
    
    # integer encode sequences of words
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(processed_data)
    unchecked_sequences = tokenizer.texts_to_sequences(processed_data)
    print('\nTotal before Sequences: %d' % len(unchecked_sequences))
    print("int sequences lengths", set([len(seq) for seq in unchecked_sequences]))    
    #     print("\nint sequences\n", sequences[:5])
    
    
    sequences = []
    for seq in unchecked_sequences:
        if len(seq) == n_seq_words:
            sequences.append(seq)
    print('\nTotal after Sequences: %d' % len(sequences))
    print("int sequences lengths", set([len(seq) for seq in sequences]))    
    

    vocab_size = len(tokenizer.word_index) + 1
    print("\nvocab_size", vocab_size)
    
    # separate into input and output
    sequences = array(sequences)
#     print("\nsequences", sequences)
    print("\nsequences.shape", sequences.shape)
    
    X, y = sequences[:,:-1], sequences[:,-1]
    print("X\n", X)
    print("y\n", y)
    
    y = to_categorical(y, num_classes=vocab_size)
    seq_length = X.shape[1]+1
#     seq_length =len(X[0])
    print("seq_length", seq_length)
    
    # ------------------------------ TRAIN MODEL -----------------------------------
    model = build_LM_A(vocab_size, seq_length, summary=True)
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model.fit(X, y, batch_size=BATCH, epochs=EPOCHS, verbose=1) 
    
    model.save(model_path)
    dump(tokenizer, open(tokenizer_path, 'wb'))
    
    return model

In [23]:
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = []
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

In [36]:
EPOCHS = 100
BATCH = 128
n_seq_words = 20
exp_count = 2

labels = {'depression': 0, 'anxiety': 1}
results = []

for label_str, label_int in labels.items():
    data = model_data[model_data["is_anxiety"] == label_int][data_column]
#     model = language_model(label_str, data, n_seq_words, EPOCHS, BATCH)
    
    model_path = '../models/{}.language_model.h5'.format(label_str)
    tokenizer_path = '../models/{}.language_model.tokenizer.pkl'.format(label_str)
    model = load_model(model_path)
    tokenizer = load(open(tokenizer_path, 'rb'))
    
    
    for i in range(exp_count):
        seed_text = data[randint(0, len(data))]
        print("seed_text\n", seed_text + '\n')
        
#         generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
        generated = generate_seq(model, tokenizer, n_seq_words, seed_text, n_seq_words)
        print("i exp:", generated)
        
        model_results = {}
        model_results["seed_text"] = seed_text
        model_results["generated"] = generated
        model_results["is_anxiety"] = label_int
        model_results["is_anxiety"] = n_seq_words
        results.append(model_results) 
    
return pd.DataFrame(results)

seed_text
 usually fall asleep lay run different communication scenario head different people play could happen last night wa different one day may wake told wake hope kid know love much hope loved wish could better hope one find hope could find another figure love depend better could depend really feeling really done life sure going next man fucking tired living

i exp: meaning regard depressed inside want chaos anyway feed fault even followed class know assume example suffering want good want understand
seed_text
 depressed year take med go therapy best follow every advice doctor feel bad time anymore functional still depressive thought motivation anything reason live doe ever get better long taking med actually started live please give hope

i exp: assure post rather want free lie someone together invalidate shes fuckkkk feel like stressed time spend different diagnosed depression mild


KeyError: 716